1. What is logic programming
2. Kanren package
3. Basic concepts:
  - terms: numbers, strings, logic variables, tuples
  - unification and reification
  - goals
  - goal generators
4. Representing knowledge
5. Example of inference

In [0]:
!pip install kanren

In [0]:
from kanren import *

Terms: constants, variables, tuples

Example of constants

In [0]:
x = Var('x')
obj1 = 'Morgen'
obj2 = 'Morg'

run(0, x, eq(obj1, obj2))

()

Example of variable

In [0]:
run(0, x, eq(x, 5))

(5,)

Logic variables are quite different than variables in procedural programming

In [0]:
y = Var()
run(0, y, eq(y, x))

In [0]:
print(x)

A variable and tuples

In [0]:
run(0, x, eq((3, 5),(x, 5)))

(3,)

Unification

We unify two similar terms like `(1, 2)` and `(1, ~x)` to form a substitution `{~x: 2}`. We say that `(1, 2)` and `(1, ~x)` unify under the substitution `{~x: 2}`. Variables may assume the value of any term.

Unify is a function that takes two terms, `u` and `v`, and returns a substitution `s`.

In [0]:
x = Var('x')
unify((x,4),(2,4))

{~x: 2}

Reification

Reify is the opposite of unify. `reify` transforms a term with logic variables like `(1, ~x)` and a substitution like `{~x: 2}` into a term without logic variables like `(1, 2)`.

In [0]:
a = var('a')
reify(x, {x: 2})

2

In [0]:
y = var('y')
reify((x, y),{x: 2, y: 3})

---
Goals - eq, membero

`run` function:
  - number of terms returned
  - variable to be assigned 
  - goal(s)

In [0]:
run(1, x, eq(x, 1))

(1,)

In [0]:
run(0, x, membero(x, (1,2,3)))

(1, 2, 3)

run has an implicit `lall` for the goals included

In [0]:
run(0, x, membero(x, (1, 2, 3)), membero(x, (2, 3, 4)))

(2, 3)

To check if a given number is a member of a given collection - write number to be checked in membero() 

In [0]:
a = 3
goal = membero(a,(4,2,3))
run(0, x, goal)

To check which numbers are in a given collection write a logic variable in membero()

In [0]:
g = run(0, x, membero(x, (1, 2, 3)), membero(x, (2, 3, 4)))
print(g)

Representing knowledge

In [0]:
parent = Relation()
marriage = Relation()
facts(parent, ("Eddard", "Arya"), ("Eddard", "Robb"), ("Rickard",  "Eddard"), ("Catlyn", "Arya"))
facts(marriage, ("Eddard", "Catlyn"))

In [0]:
run(2, x, parent(x, "Arya"))

('Eddard', 'Catlyn')

In [0]:
run(2, x, parent('Eddard', x))

('Arya', 'Robb')

In [0]:
run(2, x, parent("Cersei", x))

()

Grandparent relation 

In [0]:
def grandparent(x, z):
  y = var()
  return conde((parent(x, y), parent(y, z)))

In [0]:
run(0, x, grandparent(x, 'Arya'))

('Rickard',)

In [0]:
run(1, x, grandparent(x, 'Arya'), (parent, x, "Robb"))

()

Example - small database of US states

In [0]:
!wget 'https://raw.githubusercontent.com/logpy/logpy/master/examples/data/adjacent-states.txt'

In [0]:
adjacent = Relation()
coastal  = Relation()


coastal_states = 'WA,OR,CA,TX,LA,MI,AL,GA,FL,SC,NC,VI,MD,DW,NJ,NY,CT,RI,MA,MN,NH'.split(',')

for state in coastal_states:        # ['NY', 'NJ', 'CT', ...]
    fact(coastal, state)            # e.g. 'NY' is coastal

with open('adjacent-states.txt') as f: # lines like 'CA,OR,NV,AZ'
    adjlist = [line.strip().split(',') for line in f
                                       if line and line[0].isalpha()]

for L in adjlist:                   # ['CA', 'OR', 'NV', 'AZ']
    head, tail = L[0], L[1:]        # 'CA', ['OR', 'NV', 'AZ']
    for state in tail:
        fact(adjacent, head, state) # e.g. 'CA' is adjacent to 'OR',
                                    #      'CA' is adjacent to 'NV', etc...

x = var()
y = var()

In [0]:
run(0, x, coastal(x))

('CA',
 'NY',
 'MD',
 'NH',
 'NJ',
 'MA',
 'MI',
 'VI',
 'DW',
 'FL',
 'OR',
 'LA',
 'SC',
 'TX',
 'NC',
 'AL',
 'MN',
 'CT',
 'WA',
 'GA',
 'RI')

In [0]:
run(0, x, adjacent('ID', x))

('MT', 'UT', 'NV', 'WY', 'WA', 'OR')

In [0]:
run(0, x, adjacent('IL', x), coastal(x))

()

In [0]:
run(0, x, (adjacent, 'AL', x), coastal(x))

('GA', 'FL')

In [0]:
run(0, x, adjacent('TN', x), adjacent('FL', x))

In [0]:
run(5, x, coastal(y), adjacent(x, y), not coastal(x))

TypeError: ignored

In [0]:
inland = Relation()

for L in adjlist:
  state = L[0]
  if run(0, x, coastal(state)) == ():
    fact(inland, state)

In [0]:
run(0, x, inland(x))

('IN',
 'VT',
 'DE',
 'IL',
 'WV',
 'WI',
 'MO',
 'UT',
 'SD',
 'AZ',
 'TN',
 'NV',
 'IA',
 'VA',
 'KS',
 'WY',
 'KY',
 'ID',
 'NE',
 'ND',
 'OH',
 'DC',
 'AK',
 'MS',
 'AR',
 'CO',
 'HI',
 'ME',
 'NM',
 'OK',
 'MT',
 'PA')

In [0]:
run(0, x, coastal(y), adjacent(x, y), inland(x))

('AZ',
 'PA',
 'VA',
 'ME',
 'DE',
 'VT',
 'WI',
 'NV',
 'AR',
 'OK',
 'MS',
 'IA',
 'ID',
 'TN',
 'WV',
 'OH',
 'ND',
 'DC',
 'IN',
 'NM',
 'SD')